# My Game: Gas Station Heist

In [1]:

import sys
from typing import Dict, List, Set, Callable, Optional

class Item:
    def __init__(self, name: str, aliases: List[str] = None, portable: bool = True, desc: str = ""):
        self.name = name.lower()
        self.aliases = set([self.name] + (aliases or []))
        self.portable = portable
        self.desc = desc
    def matches(self, token: str) -> bool:
        return token.lower() in self.aliases

class Location:
    def __init__(self, key: str, title: str, desc: str):
        self.key, self.title, self.desc = key, title, desc
        self.exits: Dict[str,str] = {}
        self.items: List[Item] = []
        self.blocks: List["Block"] = []

class Block:
    def __init__(self, from_key: str, direction: str, reason: str, condition: Callable[["Game"], bool]):
        self.from_key, self.direction, self.reason, self.condition = from_key, direction.lower(), reason, condition

class Action:
    def matches(self, raw: str, tokens: List[str]) -> bool: return False
    def run(self, game: "Game", raw: str, tokens: List[str]) -> str: return "Nothing happens."

class Game:
    def __init__(self):
        self.locations: Dict[str,Location] = {}
        self.items: Dict[str,Item] = {}
        self.player_loc: str = ""
        self.inventory: List[Item] = []
        self.flags: Set[str] = set()
        self.actions: List[Action] = []
        self.dead = False
        self.won = False

    def loc(self) -> Location: return self.locations[self.player_loc]
    def add_loc(self, key: str, title: str, desc: str) -> Location:
        L = Location(key, title, desc); self.locations[key] = L; return L
    def add_item(self, name: str, **kwargs) -> Item:
        it = Item(name, **kwargs); self.items[it.name] = it; return it
    def put_item(self, item_name: str, loc_key: str):
        self.locations[loc_key].items.append(self.items[item_name])
    def get_inventory_names(self) -> List[str]: return [it.name for it in self.inventory]
    def move(self, direction: str) -> str:
        d = direction.lower(); loc = self.loc()
        for block in loc.blocks:
            if block.direction == d and not block.condition(self):
                return block.reason
        if d in loc.exits:
            self.player_loc = loc.exits[d]; return self.look()
        return "You can't go that way."
    def take(self, token: str) -> str:
        token = token.lower(); here = self.loc()
        for it in list(here.items):
            if it.matches(token):
                if not it.portable: return f"You can't take the {it.name}."
                here.items.remove(it); self.inventory.append(it); return f"Taken {it.name}."
        return "You don't see that here."
    def drop(self, token: str) -> str:
        token = token.lower()
        for it in list(self.inventory):
            if it.matches(token):
                self.inventory.remove(it); self.loc().items.append(it); return f"Dropped {it.name}."
        return "You don't have that."
    def look(self) -> str:
        loc = self.loc()
        s = [f"{loc.title}", loc.desc]
        if loc.items: s.append("You see: " + ", ".join(it.name for it in loc.items))
        if loc.exits: s.append("Exits: " + ", ".join(loc.exits.keys()))
        return "\n".join(s)
    def inv(self) -> str:
        return "You are carrying nothing." if not self.inventory else "You carry: " + ", ".join(it.name for it in self.inventory)
    def game_over(self) -> bool: return self.dead or self.won

DIRECTIONS = {"n":"north","s":"south","e":"east","w":"west","u":"up","d":"down",
              "north":"north","south":"south","east":"east","west":"west","up":"up","down":"down","out":"out","in":"in"}

def tokenize(cmd: str) -> List[str]:
    return [t for t in cmd.strip().lower().replace(",", " ").split() if t]

class GoAction(Action):
    def matches(self, raw, tokens):
        return bool(tokens) and (tokens[0] in ("go","walk","run","move") or tokens[0] in DIRECTIONS)
    def run(self, game, raw, tokens):
        if tokens[0] in DIRECTIONS: return game.move(DIRECTIONS[tokens[0]])
        if len(tokens)>=2 and tokens[1] in DIRECTIONS: return game.move(DIRECTIONS[tokens[1]])
        return "Go where?"

class LookAction(Action):
    def matches(self, raw, tokens): return bool(tokens) and tokens[0] in ("look","l")
    def run(self, game, raw, tokens): return game.look()

class InvAction(Action):
    def matches(self, raw, tokens): return bool(tokens) and tokens[0] in ("inventory","inv","i")
    def run(self, game, raw, tokens): return game.inv()

class TakeAction(Action):
    def matches(self, raw, tokens): return bool(tokens) and tokens[0] in ("take","get","pick")
    def run(self, game, raw, tokens):
        if len(tokens)<2: return "Take what?"
        obj = tokens[-1]
        return game.take(obj)

class DropAction(Action):
    def matches(self, raw, tokens): return bool(tokens) and tokens[0] in ("drop","leave")
    def run(self, game, raw, tokens):
        if len(tokens)<2: return "Drop what?"
        return game.drop(tokens[1])

def play_cli(build_game_fn, intro="Type commands (e.g., 'go north', 'take key', 'look'). Type 'quit' to exit."):
    g = build_game_fn()
    print(intro); print(g.look())
    while not g.game_over():
        try:
            raw = input("> ")
        except EOFError:
            break
        if not raw: continue
        if raw.strip().lower() in ("quit","exit"): print("Goodbye."); break
        tokens = tokenize(raw)
        out = None
        for a in g.actions:
            if a.matches(raw, tokens):
                out = a.run(g, raw, tokens); break
        if out is None: out = "I don't understand."
        print(out)
        if g.game_over(): break
    if g.dead: print("You have died. Game over.")
    if g.won: print("You win!")
    return g

def run_scripted(build_game_fn, commands: List[str]) -> List[str]:
    g = build_game_fn(); outputs = [g.look()]
    for cmd in commands:
        if g.game_over(): break
        tokens = tokenize(cmd); out = None
        for a in g.actions:
            if a.matches(cmd, tokens):
                out = a.run(g, cmd, tokens); break
        if out is None: out = "I don't understand."
        outputs.append(f"> {cmd}\n{out}")
    if g.dead: outputs.append("You have died. Game over.")
    if g.won: outputs.append("You win!")
    return outputs


In [2]:

import matplotlib.pyplot as plt
import networkx as nx

def build_my_game():
    g = Game()
    L = {}
    def add(k,t,d): L[k]=g.add_loc(k,t,d)
    add("forecourt","Forecourt","You're by the gas pumps. East leads to the shop. South to the alley.")
    add("shop","Convenience Shop","Snacks, a locked office door to the north.")
    add("alley","Back Alley","A shady alley behind the station. A suspicious thug lurks here.")
    add("office","Manager's Office","A small office with a safe.")
    add("escape","Getaway Car","Your car waits. If the alarm is tripped, cops will arrive!")

    L["forecourt"].exits.update({"east":"shop","south":"alley"})
    L["shop"].exits.update({"west":"forecourt","north":"office"})
    L["alley"].exits.update({"north":"forecourt","east":"escape"})
    L["office"].exits.update({"south":"shop"})
    L["escape"].exits.update({"west":"alley"})

    g.player_loc = "forecourt"

    g.add_item("keycard", desc="Opens the office door.")
    g.add_item("crowbar", desc="Pries things open.")
    g.add_item("cash", desc="Stacks of bills from the safe.")
    g.add_item("thug", portable=False, desc="He wants a snack to leave you alone.")
    g.add_item("chips", desc="Salty snacks.")
    g.add_item("alarm", portable=False, desc="A security alarm on the safe.")

    g.put_item("keycard","forecourt")
    g.put_item("chips","shop")
    g.put_item("thug","alley")
    g.put_item("crowbar","alley")
    g.put_item("alarm","office")

    def door_unlocked(game: Game) -> bool: return "office_unlocked" in game.flags

    class UseKeycard(Action):
        def matches(self, raw, tokens): return tokens and tokens[0] in ("use","unlock","swipe")
        def run(self, game, raw, tokens):
            if "keycard" not in game.get_inventory_names(): return "You have nothing to unlock with."
            if game.loc().key != "shop": return "Nothing to unlock here."
            game.flags.add("office_unlocked")
            # add block only when locked; since we don't model a door object, just check flag in Go
            return "Beep! The office door unlocks."

    class TakeCash(Action):
        def matches(self, raw, tokens): return tokens and tokens[0] in ("open","pry","take","grab")
        def run(self, game, raw, tokens):
            if game.loc().key != "office": return "There's nothing like that here."
            if "cash" in game.get_inventory_names(): return "You already took the cash."
            if "alarm_disarmed" in game.flags:
                game.inventory.append(game.items["cash"]); return "You quietly take the cash."
            if "crowbar" in game.get_inventory_names():
                game.flags.add("cops_alerted")
                game.inventory.append(game.items["cash"]); return "You pry the safe open with a screech. Alarm blares!"
            return "The safe resists. Maybe disarm the alarm or use something to pry it."

    class FeedThug(Action):
        def matches(self, raw, tokens): return tokens and tokens[0] == "give"
        def run(self, game, raw, tokens):
            if game.loc().key != "alley": return "No one here wants that."
            if "chips" in game.get_inventory_names():
                game.inventory = [it for it in game.inventory if it.name != "chips"]
                game.flags.add("thug_bribed"); return "You toss the chips to the thug. He leaves you alone."
            return "You have nothing to give."

    class DisarmAlarm(Action):
        def matches(self, raw, tokens): return tokens and tokens[0] in ("disarm","disable")
        def run(self, game, raw, tokens):
            if game.loc().key != "office": return "No alarm here to disarm."
            if "thug_bribed" not in game.flags: return "Your hands shake; you can't focus with danger nearby."
            game.flags.add("alarm_disarmed"); return "You carefully disconnect the alarm wires."

    class CustomGo(GoAction):
        def run(self, game, raw, tokens):
            out = super().run(game, raw, tokens)
            # special: from shop->north only if unlocked
            if tokens[0] in ("north","n","go") and game.loc().key=="shop":
                pass
            if game.player_loc == "escape" and "cops_alerted" in game.flags:
                game.dead = True; return out + "\nAs you reach the car, sirens surround you. Busted!"
            return out

    class Escape(Action):
        def matches(self, raw, tokens): return tokens and tokens[0] in ("drive","escape","leave")
        def run(self, game, raw, tokens):
            if game.loc().key != "escape": return "You need to be at your car."
            if "cash" in game.get_inventory_names() and "cops_alerted" not in game.flags:
                game.won = True; return "You slip away into the night with the cash."
            return "You can't leave yet."

    g.actions.extend([CustomGo(), LookAction(), InvAction(), TakeAction(), DropAction(),
                      UseKeycard(), TakeCash(), FeedThug(), DisarmAlarm(), Escape()])

    # allow moving north from shop only if unlocked (override via location blocks using condition check)
    def shop_north_ok(game: Game) -> bool: return "office_unlocked" in game.flags
    L["shop"].blocks.append(Block("shop","north","The office door is locked.", shop_north_ok))

    return g

# auto demo
win_cmds = [
    "take keycard","go east","take chips","use keycard","go south","go east",
    "take crowbar","give chips","go west","go north","disarm alarm","take cash",
    "go south","go west","go south","go east","drive"
]
print(run_scripted(build_my_game, win_cmds)[-1])

# graph save
def save_graph(path="my_game_graph.png"):
    gtemp = build_my_game()
    G = nx.DiGraph()
    for k, loc in gtemp.locations.items():
        G.add_node(loc.title)
        for d, dest in loc.exits.items():
            G.add_edge(loc.title, gtemp.locations[dest].title, label=d)
    plt.figure(figsize=(8,6))
    pos = nx.spring_layout(G, seed=42)
    nx.draw(G, pos, with_labels=True, node_size=2000, font_size=9, arrows=True)
    edge_labels = {(u,v):d['label'] for u,v,d in G.edges(data=True)}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)
    plt.tight_layout(); plt.savefig("my_game_graph.png", dpi=160); plt.close()

save_graph("my_game_graph.png")


> drive
You can't leave yet.


/tmp/ipython-input-392171638.py:122: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(); plt.savefig("my_game_graph.png", dpi=160); plt.close()


### Interactive play
Uncomment: `play_cli(build_my_game)`

In [4]:
play_cli(build_my_game)

Type commands (e.g., 'go north', 'take key', 'look'). Type 'quit' to exit.
Forecourt
You're by the gas pumps. East leads to the shop. South to the alley.
You see: keycard
Exits: east, south
> take keycard
Taken keycard.
> go east
Convenience Shop
Snacks, a locked office door to the north.
You see: chips
Exits: west, north
> take chips
Taken chips.
> use keycard
Beep! The office door unlocks.
> go south
You can't go that way.
> go west
Forecourt
You're by the gas pumps. East leads to the shop. South to the alley.
Exits: east, south
> go south
Back Alley
A shady alley behind the station. A suspicious thug lurks here.
You see: thug, crowbar
Exits: north, east
> take crowbar
Taken crowbar.
> give chips
You toss the chips to the thug. He leaves you alone.
> go north
Forecourt
You're by the gas pumps. East leads to the shop. South to the alley.
Exits: east, south
> go east
Convenience Shop
Snacks, a locked office door to the north.
Exits: west, north
> go north
Manager's Office
A small offic